# Connect Spark with Redis Cloud
Spark and Redis together unlock powerful capabilities for data professionals. This guide demonstrates how to integrate these technologies for enhanced analytics, real-time processing, and machine learning applications.

In this hands-on notebook, you'll learn how to make efficient use of Redis data structures alongside Spark's distributed computing framework. You'll see firsthand how to extract data from Redis, process it in Spark, and write results back to Redis for application use. Key topics include:
1. Setting up the Spark-Redis connector in Databricks
2. Writing data to Redis from Spark
3. Reading data from Redis for application access

## Databricks Cluster Setup with Redis Connector

1. Set up a new Databricks cluster
1. Go to the cluster's **Libraries** section
1. Select **Install New**
1. Choose **Maven** as your source and click **Search Packages**
1. Enter `redis-spark-connector` and select `com.redis:redis-spark-connector:x.y.z`
1. Finalize by clicking **Install** <br/>
Want to explore the connector's full capabilities? Check the [detailed documentation](https://redis-field-engineering.github.io/redis-spark)

## Setting Up Redis Cloud Environment

Redis Cloud offers a fully-managed Redis service ideal for this integration. Follow these steps:

1. Register for a [Redis Cloud account](https://redis.io/cloud/)
1. Follow the [quickstart guide](https://redis.io/docs/latest/operate/rc/rc-quickstart/) to create a free tier database
1. From your Redis Cloud database dashboard in the Configuration tab click on Import. In the dialog window select Google Cloud Storage as the source type, paste the following in the source path: [gs://jrx/nobels.rdb](gs://jrx/nobels.rdb), and finally click Import.
1. Examine the keys you've imported using **Redis Insight**, Redis' visual data browser. From your Redis Cloud database dashboard, click on **Redis Insight** and explore the data we imported in the previous step.  

## Configuring Spark with Redis Connection Details

1. From your Redis Cloud database dashboard, find your connection endpoint under **Connect**. The string follows this pattern: `redis://<user>:<pass>@<host>:<port>`
1. In Databricks, open your cluster settings and locate **Advanced Options**. Under **Spark** in the **Spark config** text area, add your Redis connection string as both `spark.redis.read.connection.uri redis://...` and `spark.redis.write.connection.uri redis://...` parameters. This configuration applies to all notebooks using this cluster. Note that it is recommended to use secrets to store sensitive Redis URIs. Refer to the [Redis Spark documentation](https://redis-field-engineering.github.io/redis-spark/#_databricks) for more details.

## Reading from Redis

To read data from Redis use the following line.

In [0]:
df = spark.read.format("redis").option("keyType", "hash").load()
display(df)

## Writing to Redis

Let's use the `df` data we imported earlier and write it back to Redis as JSON. Refresh **Redis Insight** and notice the new JSON keys prefixed with `spark:nobel`.

In [0]:
df.write.format("redis").option("type", "json").option("keyspace", "spark:nobel").option("key", "id").mode("append").save()

## Reading from Redis in Streaming Mode

The following code reads data from the Redis stream `nobels`, appending data to a streaming in-memory dataframe.

In [0]:
streamDf = spark.readStream.format("redis").option("type", "stream").option("streamKey", "nobels").load()
query = streamDf.writeStream.format("memory").queryName("nobels").outputMode("append").trigger(processingTime="1 second").start()

import time
time.sleep(3)
display(spark.sql("SELECT * FROM nobels"))

With the previously created streaming dataframe still running we can add data to the stream and see the dataframe receiving that new data. In **Redis Insight** change the "All Key Types" filter to only show keys of **Stream** type. Double-click on the `nobels` stream and click `New Entry`. Add the following fields: `category`: `physics`, `id`: `123`, `share`: `1`, `year`: `2025`, `firstName`, `lastName`, and `motivation`. Hit `Save` and run the query again. You should now see your entry in at the bottom of the table:

In [0]:
display(spark.sql("SELECT * FROM nobels"))

## Writing to Redis in Streaming Mode

We can also write to Redis in streaming mode.
1. Replace `<catalog>`, `<schema>`, and `<volume>` with names for a Unity Catalog volume.

In [0]:
catalog = "<catalog>"
schema = "<schema>"
volume = "<tutorial>"

path_volume = f"/Volumes/{catalog}/{schema}/{volume}"
checkpoint_dir = f"{path_volume}/mycp"
dbutils.fs.mkdirs(checkpoint_dir)

streamDf.writeStream.format("redis").outputMode("append") \
                 .option("type", "hash") \
                 .option("keyspace", "spark:nobel") \
                 .option("key", "id") \
                 .option("checkpointLocation", checkpoint_dir) \
                 .start()


In **Redis Insight** select keys with the pattern `spark:nobel:*`. You should see hashes corresponding to the entries in the `nobels` that we used previously. If you add other entries to the stream like we did in the *Reading from Redis in Streaming Mode* section, you will them reflected in that `spark:nobel` keyspace.